In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/group-1.3/

%load_ext autoreload
%autoreload 2

/content/drive/MyDrive/group-1.3


In [ ]:
!pip install mittens transformers
import nltk
nltk.download('words')

# student

In [ ]:
%cd /content/drive/MyDrive/group-1.3/LeverageJustAFewKeywords/

/content/drive/MyDrive/group-1.3/LeverageJustAFewKeywords


## Trainer 

In [ ]:
from train import Trainer

In [ ]:
hparams = {
    'domain': 'organic',
    'experiment_mode': 'one-time', # 'multi-times', 'debug'
    'lr': 5e-4,
    'batch_size': 1024,
    'inner_iter': 5,
    'epochs': 6,
    'gpu': '1',
    'student': {
        'pretrained': 'word2vec',
        'wv_path': '../wv/w2v_corpus_wotf1_tuned.bin',
        'wv_mode': 'tuned',     # 'pretrained'
        'pretrained_dim': 300,
        'num_aspect': 6,
        'freeze_emb': 1,
        'dropout': 0.5,
        'weight_decay': 0.1,
    },
    'data_dir': '../processed/',
    'output_dir': '../experiments/',
    'general_asp': 0,
    'maxlen': 40
}


In [ ]:
trainer = Trainer(hparams, 'cuda')

INFO:root:loading dataset...
INFO:root:[organic] dataset from following files: ../processed/organic_train.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_train_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] dataset_size: 438241
INFO:root:[organic] number of aspects: 6
INFO:root:[organic] number of unique seed words: 168
INFO:root:[organic] test dataset from following files: ../processed/annotated_test_coarse.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_test_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] test dataset size: 4687
INFO:root:loading model...
INFO:gensim.models.utils_any2vec:loading projection weights from ../wv/w2v_corpus_wotf1_tuned.bin
INFO:gensim.models.utils_any2vec:loaded (25840, 300) matrix from ../wv/w2v_corpus_wotf1_tuned.bin
INFO:root:[organic] load pre-defined word vectors from ../wv/w2v_corpus_wotf1_tuned.bin


In [ ]:
trainer.train()

## parser.py

In [ ]:
! python parser.py --domain bags_and_cases --experiment_mode one-time --lr 5e-4 --batch_size 512 --pretrained word2vec --wv_path ../wv/w2v_oposum --wv_mode tuned \
            --num_aspects 9 --data_dir ../processed/ --output_dir ../experiments/ --general_asp 4

In [ ]:
! python parser.py --domain organic --experiment_mode one-time --lr 5e-4 --batch_size 512 --pretrained word2vec --wv_path ../wv/w2v_corpus_wotf1_wostw_tuned.bin --wv_mode tuned \
            --num_aspects 6 --data_dir ../processed/ --output_dir ../experiments/ --general_asp 0

2021-07-21 15:02:38.947857: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Now GPU: Tesla P100-PCIE-16GB
INFO:root:loading dataset...
INFO:root:[organic] dataset from following files: ../processed/organic_train.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_train_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] dataset_size: 438241
INFO:root:[organic] number of aspects: 6
INFO:root:[organic] number of unique seed words: 168
INFO:root:[organic] test dataset from following files: ../processed/annotated_test_coarse.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_test_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] test dataset size: 4687
INFO:root:loading model...
INFO:gensim.models.utils_any2vec:loading projection weights from ../wv/w2v_corpus_wotf1_wostw_tuned.bin
INFO:gensim.models.utils_any2vec:loaded (25694, 300) matrix from ..

In [ ]:
! python parser.py --domain organic --experiment_mode multi-times --lr 0.005 --batch_size 512 --inner_iter 5 --epochs 4 --pretrained word2vec --wv_path ../wv/w2v_corpus_wotf1_wostw_tuned.bin --wv_mode tuned \
            --num_aspect 6 --dropout 0.4 --weight_decay 0.01 --data_dir ../processed/ --output_dir ../experiments/ --general_asp 0 

2021-07-25 11:06:38.163147: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Now GPU: Tesla P100-PCIE-16GB
experiments setting: {'domain': 'organic', 'experiment_mode': 'multi-times', 'lr': 0.005, 'batch_size': 512, 'inner_iter': 5, 'epochs': 6, 'gpu': '1', 'student': {'pretrained': 'word2vec', 'wv_path': '../wv/w2v_corpus_wotf1_wostw_tuned.bin', 'wv_mode': 'tuned', 'pretrained_dim': 300, 'num_aspect': 6, 'freeze_emb': 1, 'dropout': 0.4, 'weight_decay': 0.01}, 'data_dir': '../processed/', 'output_dir': '../experiments/', 'general_asp': 0, 'maxlen': 40}
INFO:root:loading dataset...
INFO:root:[organic] dataset from following files: ../processed/organic_train.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_train_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] dataset_size: 438241
INFO:root:[organic] number of aspects: 6
INFO:root:[organic] number of unique seed words: 168
INFO:

# Teacher Prediction


In [ ]:
%cd /content/drive/MyDrive/group-1.3/LeverageJustAFewKeywords/

/content/drive/MyDrive/group-1.3/LeverageJustAFewKeywords


In [ ]:
!pip install mittens transformers
import nltk
nltk.download('words')

In [ ]:
import torch
from sklearn.metrics import f1_score

from train import Trainer
from utils import *
from config import hparams

In [ ]:
hparams['domain'] = 'organic'
hparams['data_dir'] = '../processed'
aspect_name = read_aspect_name('../processed/organic_aspect_name_coarse.txt')
hparams['general_asp'] = aspect_name.index('general')
hparams['student']['num_aspect'] = len(aspect_name)
hparams['student']['wv_path'] = '../wv/w2v_corpus_wotf1_tuned.bin'
trainer = Trainer(hparams, 'cuda')

INFO:root:loading dataset...
INFO:root:[organic] dataset from following files: ../processed/organic_train.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_train_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] dataset_size: 438241
INFO:root:[organic] number of aspects: 6
INFO:root:[organic] number of unique seed words: 168
INFO:root:[organic] test dataset from following files: ../processed/annotated_test_coarse.json, ../processed/organic_vocab_w2v.txt, ../processed/organic_test_supplement_w2v.pkl, ../processed/seedwords_30_coarse.txt
INFO:root:[organic] test dataset size: 4687
INFO:root:loading model...
INFO:gensim.models.utils_any2vec:loading projection weights from ../wv/w2v_corpus_wotf1_tuned.bin
INFO:gensim.models.utils_any2vec:loaded (25840, 300) matrix from ../wv/w2v_corpus_wotf1_tuned.bin
INFO:root:[organic] load pre-defined word vectors from ../wv/w2v_corpus_wotf1_tuned.bin


In [ ]:
score = []
result = []
ground = []
for batch in trainer.test_loader:
    idx, bow, labels, actual_len = batch
    bow = bow.cuda()
    labels = labels.cuda()
    trainer.z = trainer.reset_z()   # no weight information
    with torch.no_grad():
        logits = trainer.teacher(bow, trainer.z)
    result.append(logits.max(-1)[1].cpu())
    ground.append(labels.max(-1)[1].cpu())
result = torch.cat(result, dim=0)
ground = torch.cat(ground, dim=0)
score = f1_score(y_true=ground.numpy(), y_pred=result.numpy(), average='micro')

In [ ]:
score

0.45743545978237676

In [ ]:
trainer.z.shape

torch.Size([6, 168])